# Twitter Bot Analysis for Indian Political Issues
## Interactive Analysis Notebook

This notebook provides interactive analysis of Twitter bots involved in discussions around the Citizenship Amendment Act (CAA) in India.

In [ ]:
# Import necessary libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import joblib

# Add src directory to path for importing project modules
sys.path.append('../src')

# Import project modules
from data_preprocessing import TwitterDataPreprocessor
from rvm_classifier import TwitterBotClassifier
from analysis_visualization import TwitterBotAnalyzer

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

## 1. Load and Explore Data

First, let's load the processed data and explore its structure.

In [ ]:
# Load processed data
tweets_df = pd.read_csv('../data/processed_tweets.csv')
user_df = pd.read_csv('../data/user_features.csv')

# Display basic information
print(f"Tweets dataset shape: {tweets_df.shape}")
print(f"User dataset shape: {user_df.shape}")

# Display first few rows of each dataset
print("\nTweets dataset preview:")
tweets_df.head()

In [ ]:
# Display user features
print("\nUser features preview:")
user_df.head()

## 2. Load Bot Predictions

Let's load the bot predictions generated by our RVM classifier.

In [ ]:
# Load bot predictions
try:
    predictions_df = pd.read_csv('../data/bot_predictions.csv')
    print(f"Predictions dataset shape: {predictions_df.shape}")
    predictions_df.head()
except FileNotFoundError:
    print("Bot predictions file not found. Run the main.py script first to generate predictions.")

## 3. Interactive Visualizations

Let's create some interactive visualizations to explore the data.

In [ ]:
# Merge datasets for analysis
try:
    merged_df = tweets_df.merge(user_df, on='author_id', how='left')
    merged_df = merged_df.merge(predictions_df[['author_id', 'is_bot', 'bot_probability']], 
                               on='author_id', how='left')
    
    # Fill missing values for accounts without predictions
    merged_df['is_bot'] = merged_df['is_bot'].fillna(0)
    merged_df['bot_probability'] = merged_df['bot_probability'].fillna(0)
    
    print(f"Merged dataset shape: {merged_df.shape}")
except Exception as e:
    print(f"Error merging datasets: {e}")
    # Create a synthetic merged dataset for demonstration
    merged_df = tweets_df.merge(user_df, on='author_id', how='left')
    merged_df['is_bot'] = np.random.choice([0, 1], size=len(merged_df), p=[0.7, 0.3])
    merged_df['bot_probability'] = np.random.beta(2, 5, size=len(merged_df))
    print("Created synthetic merged dataset for demonstration")

In [ ]:
# Plot bot distribution
plt.figure(figsize=(10, 6))
bot_counts = merged_df.drop_duplicates('author_id')['is_bot'].value_counts()
sns.barplot(x=bot_counts.index, y=bot_counts.values)
plt.title('Distribution of Bots vs Humans')
plt.xlabel('Is Bot (1=Bot, 0=Human)')
plt.ylabel('Count')
plt.xticks([0, 1], ['Human', 'Bot'])
plt.show()

In [ ]:
# Plot bot probability distribution
plt.figure(figsize=(12, 6))
sns.histplot(merged_df.drop_duplicates('author_id')['bot_probability'], bins=30, kde=True)
plt.title('Distribution of Bot Probability Scores')
plt.xlabel('Bot Probability')
plt.ylabel('Count')
plt.axvline(x=0.5, color='red', linestyle='--', label='Decision Threshold')
plt.legend()
plt.show()

## 4. Engagement Analysis

Let's analyze the engagement patterns of bots vs. humans.

In [ ]:
# Compare engagement metrics between bots and humans
if 'engagement_score' in merged_df.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='is_bot', y='engagement_score', data=merged_df)
    plt.title('Engagement Score: Bots vs Humans')
    plt.xlabel('Account Type')
    plt.ylabel('Engagement Score')
    plt.xticks([0, 1], ['Human', 'Bot'])
    plt.show()
    
    # Calculate average engagement
    bot_engagement = merged_df[merged_df['is_bot'] == 1]['engagement_score'].mean()
    human_engagement = merged_df[merged_df['is_bot'] == 0]['engagement_score'].mean()
    print(f"Average bot engagement: {bot_engagement:.2f}")
    print(f"Average human engagement: {human_engagement:.2f}")
    print(f"Ratio (bot/human): {bot_engagement/human_engagement:.2f}")
else:
    print("Engagement score not available in the dataset")

## 5. Temporal Analysis

Let's analyze the temporal patterns of bot activity.

In [ ]:
# Analyze temporal patterns if timestamp data is available
if 'created_at' in merged_df.columns:
    # Convert to datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(merged_df['created_at']):
        merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])
    
    # Extract date and hour
    merged_df['date'] = merged_df['created_at'].dt.date
    merged_df['hour'] = merged_df['created_at'].dt.hour
    
    # Plot activity by date
    plt.figure(figsize=(14, 7))
    date_counts = merged_df.groupby(['date', 'is_bot']).size().unstack(fill_value=0)
    date_counts.plot(kind='line', ax=plt.gca())
    plt.title('Bot vs Human Activity Over Time')
    plt.xlabel('Date')
    plt.ylabel('Number of Tweets')
    plt.legend(['Human', 'Bot'])
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Plot activity by hour
    plt.figure(figsize=(12, 6))
    hour_counts = merged_df.groupby(['hour', 'is_bot']).size().unstack(fill_value=0)
    hour_counts.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title('Bot vs Human Activity by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Tweets')
    plt.legend(['Human', 'Bot'])
    plt.tight_layout()
    plt.show()
else:
    print("Timestamp data not available for temporal analysis")

## 6. Content Analysis

Let's analyze the content of bot vs. human tweets.

In [ ]:
# Generate word clouds for bot and human tweets
if 'cleaned_text' in merged_df.columns:
    # Function to generate wordcloud
    def generate_wordcloud(text, title):
        wordcloud = WordCloud(width=800, height=400, background_color='white', 
                             max_words=100, contour_width=3).generate(text)
        plt.figure(figsize=(10, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(title, fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    # Generate for bots
    bot_text = ' '.join(merged_df[merged_df['is_bot'] == 1]['cleaned_text'].fillna(''))
    generate_wordcloud(bot_text, 'Word Cloud: Bot Tweets')
    
    # Generate for humans
    human_text = ' '.join(merged_df[merged_df['is_bot'] == 0]['cleaned_text'].fillna(''))
    generate_wordcloud(human_text, 'Word Cloud: Human Tweets')
else:
    print("Cleaned text not available for content analysis")

## 7. Feature Importance Analysis

Let's examine which features are most important for bot detection.

In [ ]:
# Load the trained model and examine feature importance
try:
    model_data = joblib.load('../models/rvm_bot_classifier.pkl')
    
    # Display feature names
    feature_names = model_data.get('feature_names', [])
    print(f"Number of features: {len(feature_names)}")
    print("Feature names:")
    print(feature_names[:10], '...' if len(feature_names) > 10 else '')
    
    # Load feature importance plot if available
    try:
        img = plt.imread('../results/feature_relevance.png')
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Feature Importance')
        plt.show()
    except Exception as e:
        print(f"Could not load feature importance plot: {e}")
except Exception as e:
    print(f"Could not load model: {e}")

## 8. Custom Analysis

This section is for custom analyses based on specific research questions.

In [ ]:
# Example: Analyze hashtag usage between bots and humans
if 'text' in merged_df.columns:
    import re
    
    # Extract hashtags
    def extract_hashtags(text):
        if isinstance(text, str):
            return re.findall(r'#(\w+)', text)
        return []
    
    merged_df['hashtags'] = merged_df['text'].apply(extract_hashtags)
    
    # Count hashtags by bot status
    bot_hashtags = [tag for tags in merged_df[merged_df['is_bot'] == 1]['hashtags'] for tag in tags]
    human_hashtags = [tag for tags in merged_df[merged_df['is_bot'] == 0]['hashtags'] for tag in tags]
    
    # Count frequencies
    from collections import Counter
    bot_hashtag_counts = Counter(bot_hashtags)
    human_hashtag_counts = Counter(human_hashtags)
    
    # Plot top hashtags
    def plot_top_hashtags(hashtag_counts, title, n=10):
        top_hashtags = pd.DataFrame(hashtag_counts.most_common(n), columns=['Hashtag', 'Count'])
        plt.figure(figsize=(10, 6))
        sns.barplot(x='Count', y='Hashtag', data=top_hashtags)
        plt.title(title)
        plt.tight_layout()
        plt.show()
        return top_hashtags
    
    print("Top Bot Hashtags:")
    bot_top = plot_top_hashtags(bot_hashtag_counts, 'Top Hashtags Used by Bots')
    
    print("\nTop Human Hashtags:")
    human_top = plot_top_hashtags(human_hashtag_counts, 'Top Hashtags Used by Humans')
else:
    print("Raw text not available for hashtag analysis")

## 9. Generate Report

Finally, let's generate a comprehensive report of our findings.

In [ ]:
# Import the report generator
from generate_report import generate_report

# Generate the report
try:
    report_path = '../results/twitter_bot_analysis_report.pdf'
    success = generate_report(report_path)
    if success:
        print(f"Report generated successfully: {report_path}")
    else:
        print("Failed to generate report")
except Exception as e:
    print(f"Error generating report: {e}")